# Mesh generation for tutorial 06, case 2b

This file generates the mesh which is used in the following examples:
* 2b_advection_diffusion_reaction

The test case is from section 5.2 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import typing

In [ ]:
import dolfinx.io
import gmsh
import mpi4py

In [ ]:
import multiphenicsx.io
import multiphenicsx.mesh

### Geometrical parameters and related quantities

In [ ]:
lcar = 0.05

### Create mesh with gmsh

In [ ]:
class GenerateRectangleLines(object):
    """Generate a rectangle."""

    def __init__(self) -> None:
        self.points = dict()
        self.lines = dict()

    def add_point(self, x: float, y: float) -> int:
        """Add a point to gmsh, if not present already."""
        key = (x, y)
        try:
            return self.points[key]
        except KeyError:
            p = gmsh.model.geo.addPoint(x, y, 0.0, lcar)
            self.points[key] = p
            return p

    def add_line(self, p0: int, p1: int) -> int:
        """Add a line to gmsh, if not present already."""
        try:
            return self.lines[(p0, p1)]
        except KeyError:
            l01 = gmsh.model.geo.addLine(p0, p1)
            self.lines[(p0, p1)] = l01
            self.lines[(p1, p0)] = -l01
            return l01

    def __call__(
        self, x_min: float, x_max: float, y_min: float, y_max: typing.Union[float, typing.List[float]]
    ) -> typing.Tuple[int]:
        """Add points and lines that define a rectangle with the provided coordinates."""
        p0 = self.add_point(x_min, y_min)
        p1 = self.add_point(x_max, y_min)
        if isinstance(y_max, list):
            p2 = [self.add_point(x_max, y) for y in y_max]
            p3 = self.add_point(x_min, y_max[-1])
        else:
            p2 = self.add_point(x_max, y_max)
            p3 = self.add_point(x_min, y_max)
        l0 = self.add_line(p0, p1)
        if isinstance(y_max, list):
            p1_p2 = [p1] + p2
            l1 = [self.add_line(p1_p2[i], p1_p2[i + 1]) for i in range(len(p2))]
            l2 = self.add_line(p2[-1], p3)
        else:
            l1 = self.add_line(p1, p2)
            l2 = self.add_line(p2, p3)
        l3 = self.add_line(p3, p0)
        return (l0, l1, l2, l3)


gmsh.initialize()
gmsh.model.add("mesh")
generate_rectangle_lines = GenerateRectangleLines()
[l0, l1, l2, l3] = generate_rectangle_lines(0.0, 1.0, 0.0, 1.0)
[l4, l5, l6, _] = generate_rectangle_lines(1.0, 2.5, 0.0, [0.3, 0.7, 1.0])
[l7, l8, l9, l10] = generate_rectangle_lines(0.2, 0.8, 0.3, 0.7)
[l11, l12, l13, l14] = generate_rectangle_lines(1.2, 2.5, 0.3, 0.7)
line_loop_rectangle_outer_left = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])
line_loop_rectangle_outer_right = gmsh.model.geo.addCurveLoop([l4, l5[0], l5[1], l5[2], l6, -l1])
line_loop_rectangle_inner_left = gmsh.model.geo.addCurveLoop([l7, l8, l9, l10])
line_loop_rectangle_inner_right = gmsh.model.geo.addCurveLoop([l11, l12, l13, l14])
rectangle_outer_left = gmsh.model.geo.addPlaneSurface(
    [line_loop_rectangle_outer_left, line_loop_rectangle_inner_left])
rectangle_outer_right = gmsh.model.geo.addPlaneSurface(
    [line_loop_rectangle_outer_right, line_loop_rectangle_inner_right])
rectangle_inner_left = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_inner_left])
rectangle_inner_right = gmsh.model.geo.addPlaneSurface([line_loop_rectangle_inner_right])
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0, l2, l3], 1)
gmsh.model.addPhysicalGroup(1, [l4, l6], 2)
gmsh.model.addPhysicalGroup(1, [l5[0], l5[1], l5[2]], 3)
gmsh.model.addPhysicalGroup(2, [rectangle_outer_left], 3)
gmsh.model.addPhysicalGroup(2, [rectangle_outer_right], 4)
gmsh.model.addPhysicalGroup(2, [rectangle_inner_left], 1)
gmsh.model.addPhysicalGroup(2, [rectangle_inner_right], 2)
gmsh.model.mesh.generate(2)

### Convert to a dolfinx mesh

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

### Save mesh, subdomains and boundaries

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "graetz_1.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)